In [5]:
import numpy as np
import scipy 
import scipy.linalg
import scipy.sparse
import scipy.sparse.linalg
import matplotlib.pylab as plt
import time
from math import sinh

#import matplotlib.pyplot as plt

# Change some default values to make plots more readable on the screen
LNWDT=2; FNT=15
plt.rcParams['lines.linewidth'] = LNWDT; plt.rcParams['font.size'] = FNT

def setup_LaplaceNeumann_xy(Ttop, Tright, nx, ny):
    """ Function that returns A matrix and b vector  of the laplace Neumann heat problem A*T=b using central differences
        and assuming dx=dy, based on numbering with respect to x-dir, e.g:
        
                1   1   1             -4  2  2  0  0  0          0
            T6  T5  T6  0              1 -4  0  2  0  0          0
            T4  T3  T4  0              1  0 -4  2  1  0          0
            T2  T1  T2  0     --> A =  0  1  1 -4  0  1    ,b =  0
                T3  T4                 0  0  1  0 -4  2         -1
                                       0  0  0  1  1 -4         -1
            
            T = [T1, T2, T3, T4, T5, T6]^T
            
        Args:
            nx(int): number of elements in each row in the grid, nx=2 in the example above
            ny(int): number of elements in each column in the grid, ny=3 in the example above
        Returns:
            A(matrix): Sparse matrix A, in the equation A*T = b
            b(array): RHS, of the equation A*t = b
    """
    
    n = (nx)*(ny) #number of unknowns
    d = np.ones(n) # diagonals
    b = np.zeros(n) #RHS
    
    d0 = d.copy()*-4
    d1_lower = d.copy()[0:-1]
    d1_upper = d1_lower.copy()
    
    dnx_lower = d.copy()[0:-nx]
    dnx_upper = dnx_lower.copy()
    
    d1_lower[nx-1::nx] = 0 # every nx element on first diagonal is zero; starting from the nx-th element
    d1_upper[nx-1::nx] = 0
    d1_upper[::nx] = 2 # every nx element on first upper diagonal is two; stating from the first element. 
                 # this correspond to all equations on border (x=0, y)
    
    dnx_upper[0:nx] = 2 # the first nx elements in the nx-th upper diagonal is two; 
                        # This correspond to all equations on border (x, y=0) 
    
    b[-nx:] = -Ttop
    b[nx-1::nx] += -Tright

    A = scipy.sparse.diags([d0, d1_upper, d1_lower, dnx_upper, dnx_lower], [0, 1, -1, nx, -nx], format='csc')
    
    return A, b

if __name__ == '__main__':
    from Visualization import plot_SurfaceNeumann_xy
    # Main program
    # Set temperature at the top
    Ttop=1
    Tright = 0.0
    xmax=1.0
    ymax=1.
     
    # Set simulation parameters
    #need hx=(1/nx)=hy=(1.5/ny)
    
    Nx = 10
    h=xmax/Nx
    Ny = int(ymax/h)
    
    A, b = setup_LaplaceNeumann_xy(Ttop, Tright, Nx, Ny)
    
    Temp = scipy.sparse.linalg.spsolve(A, b)
    
    plt.plot(Temp, Ttop, Tright, xmax, ymax, Nx, Ny)

#    figfile='LaPlace_vNeumann.png'
#    plt.savefig(figfile, format='png',transparent=True)
    plt.show()

ModuleNotFoundError: No module named 'Visualization'

In [ ]:
for n in range(Nx*Ny):
            j = n//Nx + 1
            i = n - Nx*(j-1)+1
            #i_c = (i)*1.61/Nx
            #j_c = (j)*1.61/Ny
            xi = xs[i-1]
            yj = ys[j-1]
            
            xi_half_b = xi-hx/2 ## x_{i-1/2}, backward half step
            xi_half_f = xi+hx/2 ## x_{i+1/2}, forward half step
            yj_half_b = yj-hy/2 ## y_{j-1/2}, backward half step
            yj_half_f = yj+hy/2 ## y_{j+1/2}, forward half step
            
            
            a = np.zeros(Nx*Ny)
            
            #a[n] = -(k(xi_half_f, yj) + k(xi_half_b, yj))/hx**2 - (k(xi, yj_half_f) + k(xi, yj_half_b))/hy**2 
            
            if check_domain(xi, yj):
                if i == Nx:
                    a[n-1] += kappa(xi, yj_half_f)/hx**2
                else:
                    a[n+1] += kappa(x_half_f, yj)
                
                if np.isclose(xi, 0.61, atol = hx) and yj > 0.61:
                    a[n] -= 12/hx
                    a[n-1] += kappa(xi, yj)/hx**2
                    b[n] -= 240/hx
                
                if j == Ny:
                    a[n-Nx] += kappa(xi, yj)/hy**2
                else:
                    a[n+Nx] += kappa(xi, y_half_f)/hy**2
                
                if np.isclose(yj, 0.61, atol = hy) and xi > 0.61:
                    a[n] -= 12/hy
                    a[n-Nx] += kappa(xi, yj)/hy**2
                    b[n] -= 240/hy
                    
                if i == 1:
                    a[n] += 50/hx
                    a[n+1] += kappa(xi, yj)/hx**2
                    b[n] -= 550/hx
                else:
                    a[n-1] += kappa(x_half_b, yj)/hx**2
                
                if j == 1:
                    a[n] += 50/hy
                    a[n+Nx] += kappa(xi, yj)/hy**2
                    b[n] -= 550/hy
                else:
                    a[n-Nx] += kappa(xi, y_half_b)/hy**2
                a[n] += -kappa(x_half_b, yj)/hx**2 - kappa(x_half_f, yj)/hx**2 - kappa(xi, y_half_f)/hy**2 - kappa(xi, y_half_b)/hy**2

                #if i_c >= 0.59 and j_c >= 0.59:
                   # print(a[n])
            else:
                a[n] = 1
                b[n] = 0

In [ ]:
plt.figure(figsize = (13,10))

ax = plt.gca()
contourplot = plt.contourf(ys, cmap = "inferno")
ticks_x = ticker.FuncFormatter(lambda x, pos: '{0:.3f}'.format(x * 1.61/Nx))
ticks_y = ticker.FuncFormatter(lambda y, pos: '{0:.3f}'.format(y * 1.61/Ny))
ax.xaxis.set_major_formatter(ticks_x)
ax.yaxis.set_major_formatter(ticks_y)
cbar = plt.colorbar(contourplot)

In [ ]:
        ## Plaster
        if (xi <= 0.02 or yj <= 0.02):
            return 0.81
        ## Isolation
        elif (xi <= 0.14) or (yj <= 0.14):
            return 0.049
        ## Porotherm
        elif (xi <= 0.59) or (yj <= 0.59):
            return 0.5
        ## Plaster 2
        elif (xi <= 1.61 and yj <= 0.61) or (xi <= 0.61 and yj <= 1.61):
            return 0.99

In [ ]:
        for n in range(Nx*Ny):
            j = n//Nx + 1
            i = n - Nx*(j-1)+1
            i_c = (i)*1.61/Nx
            j_c = (j)*1.61/Ny
            if i_c >= 0.59 and j_c >= 0.59:
                M[j-1][i-1] = np.nan